In [52]:
import geopandas as gpd
from geopandas import GeoDataFrame
import networkx as nx
import pandas as pd
import contextily as ctx
import pathlib
import numpy as np
from matplotlib import pyplot as plt
from shapely.geometry import Point, Polygon, LineString

In [31]:
import queue

SiteNumber                                                       09054.A
Type                                                              AIRPORT
LocationID                                                           '1B9
Region                                                                      ANE
geometry                     POINT (-71.19777777777779 42.00083333333333)
index                                                                  39

SiteNumber                                                           09304.02A
Type                                                                    AIRPORT
LocationID                                                                 'CEF
Region                                                                      ANE
geometry                           POINT (-72.53478430555555 42.19401497222222)
index                                                                  55

SiteNumber                                                       13434.1*A
Type                                                               AIRPORT
LocationID                                                            '8B2
Region                                                                 ANE
geometry                      POINT (-71.54674869444445 44.26406408333333)
index                                                                  153

In [44]:
game_grid = gpd.read_file('game_grid_2021.geojson')
game_grid = game_grid.drop_duplicates(subset=['MGRS'])

In [45]:
game_grid = game_grid.reset_index().drop(['index'], axis=1)
game_grid

,EASTING,NORTHING,kmSQ_ID,GZD,Shape_Leng,MGRS,MGRS_10km,transport_score,geometry
0,596000mE,4984000mN,WQ,18T,400000.000000,18TWQ9684,18TWQ98,1,"POLYGON ((-73.76921 45.00282, -73.78190 45.002..."
1,597000mE,4984000mN,WQ,18T,400000.000000,18TWQ9784,18TWQ98,1,"POLYGON ((-73.75653 45.00268, -73.76921 45.002..."
2,598000mE,4984000mN,WQ,18T,400000.000000,18TWQ9884,18TWQ98,1,"POLYGON ((-73.74384 45.00254, -73.75653 45.002..."
3,599000mE,4984000mN,WQ,18T,400000.000000,18TWQ9984,18TWQ98,1,"POLYGON ((-73.73116 45.00240, -73.74384 45.002..."
4,579000mE,4983000mN,WQ,18T,400000.000000,18TWQ7983,18TWQ78,1,"POLYGON ((-73.98504 44.99594, -73.99773 44.996..."
...,...,...,...,...,...,...,...,...,...
196817,282000mE,4558000mN,BF,19T,0.041828,19TBF8258,19TBF85,1,"POLYGON ((-71.58574 41.14431, -71.59765 41.144..."
196818,283000mE,4558000mN,BF,19T,0.041828,19TBF8358,19TBF85,1,"POLYGON ((-71.57384 41.14458, -71.58574 41.144..."
196819,284000mE,4558000mN,BF,19T,0.041828,19TBF8458,19TBF85,1,"POLYGON ((-71.56193 41.14484, -71.57384 41.144..."
196820,285000mE,4558000mN,BF,19T,0.041828,19TBF8558,19TBF85,1,"POLYGON ((-71.55003 41.14511, -71.56193 41.144..."


In [46]:
game_grid_copy = game_grid.copy()
game_grid_copy['geometry'] = game_grid.to_crs('epsg:3857').buffer(10).to_crs('epsg:4326')
game_grid_copy

,EASTING,NORTHING,kmSQ_ID,GZD,Shape_Leng,MGRS,MGRS_10km,transport_score,geometry
0,596000mE,4984000mN,WQ,18T,400000.000000,18TWQ9684,18TWQ98,1,"POLYGON ((-73.76912 45.00281, -73.76913 45.002..."
1,597000mE,4984000mN,WQ,18T,400000.000000,18TWQ9784,18TWQ98,1,"POLYGON ((-73.75644 45.00268, -73.75644 45.002..."
2,598000mE,4984000mN,WQ,18T,400000.000000,18TWQ9884,18TWQ98,1,"POLYGON ((-73.74375 45.00254, -73.74375 45.002..."
3,599000mE,4984000mN,WQ,18T,400000.000000,18TWQ9984,18TWQ98,1,"POLYGON ((-73.73107 45.00240, -73.73107 45.002..."
4,579000mE,4983000mN,WQ,18T,400000.000000,18TWQ7983,18TWQ78,1,"POLYGON ((-73.98495 44.99594, -73.98495 44.995..."
...,...,...,...,...,...,...,...,...,...
196817,282000mE,4558000mN,BF,19T,0.041828,19TBF8258,19TBF85,1,"POLYGON ((-71.58565 41.14431, -71.58565 41.144..."
196818,283000mE,4558000mN,BF,19T,0.041828,19TBF8358,19TBF85,1,"POLYGON ((-71.57375 41.14458, -71.57375 41.144..."
196819,284000mE,4558000mN,BF,19T,0.041828,19TBF8458,19TBF85,1,"POLYGON ((-71.56184 41.14484, -71.56184 41.144..."
196820,285000mE,4558000mN,BF,19T,0.041828,19TBF8558,19TBF85,1,"POLYGON ((-71.54994 41.14511, -71.54994 41.145..."


In [50]:
edges = gpd.sjoin(game_grid, game_grid_copy, how='inner', op='intersects')
edges

,EASTING_left,NORTHING_left,kmSQ_ID_left,GZD_left,Shape_Leng_left,MGRS_left,MGRS_10km_left,transport_score_left,geometry,index_right,EASTING_right,NORTHING_right,kmSQ_ID_right,GZD_right,Shape_Leng_right,MGRS_right,MGRS_10km_right,transport_score_right
0,596000mE,4984000mN,WQ,18T,400000.000000,18TWQ9684,18TWQ98,1,"POLYGON ((-73.76921 45.00282, -73.78190 45.002...",22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1
1,597000mE,4984000mN,WQ,18T,400000.000000,18TWQ9784,18TWQ98,1,"POLYGON ((-73.75653 45.00268, -73.76921 45.002...",22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1
2,598000mE,4984000mN,WQ,18T,400000.000000,18TWQ9884,18TWQ98,1,"POLYGON ((-73.74384 45.00254, -73.75653 45.002...",22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1
21,596000mE,4983000mN,WQ,18T,400000.000000,18TWQ9683,18TWQ98,1,"POLYGON ((-73.76941 44.99382, -73.78209 44.993...",22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1
22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1,"POLYGON ((-73.75672 44.99368, -73.76941 44.993...",22,597000mE,4983000mN,WQ,18T,400000.000000,18TWQ9783,18TWQ98,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196821,286000mE,4558000mN,BF,19T,0.041829,19TBF8658,19TBF85,1,"POLYGON ((-71.53813 41.14537, -71.55003 41.145...",196820,285000mE,4558000mN,BF,19T,0.041828,19TBF8558,19TBF85,1
196814,285000mE,4559000mN,BF,19T,0.041832,19TBF8559,19TBF85,2,"POLYGON ((-71.55038 41.15411, -71.56229 41.153...",196821,286000mE,4558000mN,BF,19T,0.041829,19TBF8658,19TBF85,1
196815,286000mE,4559000mN,BF,19T,0.041832,19TBF8659,19TBF85,1,"POLYGON ((-71.53847 41.15437, -71.55038 41.154...",196821,286000mE,4558000mN,BF,19T,0.041829,19TBF8658,19TBF85,1
196820,285000mE,4558000mN,BF,19T,0.041828,19TBF8558,19TBF85,1,"POLYGON ((-71.55003 41.14511, -71.56193 41.144...",196821,286000mE,4558000mN,BF,19T,0.041829,19TBF8658,19TBF85,1


In [53]:
graph = nx.Graph()
for i, r in game_grid.iterrows():
    graph.add_node(r['MGRS'])

In [54]:
count = 0
for i, r in edges.iterrows():
    if(count % 100000 == 0):
        print(count)
    
    node1 = r['MGRS_left']
    node2 = r['MGRS_right']

    edge_weight = 20. / (r['transport_score_left'] + r['transport_score_right'])
    if(node1 != node2):
        graph.add_edge(node1, node2, weight=edge_weight)
    count += 1

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000


In [13]:
dist = GeoDataFrame(columns = ['geometry'])
dist['geometry'] = [Point(-71.19777777777779, 42.00083333333333), Point(-72.53478430555555, 42.19401497222222), Point(-71.54674869444445, 44.26406408333333)]
dist = dist.set_crs('epsg:4326')

In [21]:
dist = gpd.sjoin(dist, game_grid, how='inner', op='within')
dist

,geometry,index_right,EASTING,NORTHING,kmSQ_ID,GZD,Shape_Leng,MGRS,MGRS_10km,transport_score
0,POINT (-71.19778 42.00083),111183,317000mE,4652000mN,CG,19T,0.042148,19TCG1752,19TCG15,10
0,POINT (-71.19778 42.00083),111184,317000mE,4652000mN,CG,19T,0.042148,19TCG1752,19TCG15,10
1,POINT (-72.53478 42.19401),96926,703000mE,4674000mN,YM,18T,297265.528077,18TYM0374,18TYM07,1
2,POINT (-71.54675 44.26406),77303,296000mE,4904000mN,BK,19T,0.043044,19TBK9604,19TBK90,10


In [24]:
dist

,geometry,index_right,EASTING,NORTHING,kmSQ_ID,GZD,Shape_Leng,MGRS,MGRS_10km,transport_score
0,POINT (-71.19778 42.00083),111183,317000mE,4652000mN,CG,19T,0.042148,19TCG1752,19TCG15,10
1,POINT (-72.53478 42.19401),96926,703000mE,4674000mN,YM,18T,297265.528077,18TYM0374,18TYM07,1
2,POINT (-71.54675 44.26406),77303,296000mE,4904000mN,BK,19T,0.043044,19TBK9604,19TBK90,10


In [29]:
hospitals_url = 'Hospitals.csv'
hospitals_df = pd.read_csv(hospitals_url)

hospitals_df = hospitals_df.fillna('')
hospitals_df = hospitals_df.loc[(hospitals_df['X']>= -74.0062751207002) & (hospitals_df['X']<= -66.93832921282291) & (hospitals_df['Y'] >= 40.995556258214776) & (hospitals_df['Y'] <= 47.46624330268622)]
hospitals_df = hospitals_df.reset_index().drop(['index'], axis=1)

loc_gdf = GeoDataFrame()
loc_gdf['geometry'] = None
for i, r in hospitals_df.iterrows():
    h_point = Point(r['X'], r['Y'])
    loc_gdf.loc[i, 'geometry'] = h_point
loc_gdf = loc_gdf.set_crs(epsg=4326)

joined_loc = gpd.sjoin(loc_gdf, game_grid, how='inner', op='within')
joined_loc = joined_loc.reset_index().drop(['index'], axis=1)

In [142]:
hosp = joined_loc.copy()

hosp["vis"] = 0
hosp["incoming"] = [set() for i in range (hosp.shape[0])]
hosp["supplies"] = 5000
hosp

,geometry,index_right,EASTING,NORTHING,kmSQ_ID,GZD,Shape_Leng,MGRS,MGRS_10km,transport_score,vis,incoming,supplies
0,POINT (-72.74062 41.70169),121303,687000mE,4619000mN,XM,18T,400000.000000,18TXM8719,18TXM81,3,0,{},5000
1,POINT (-71.05394 42.06302),110181,330000mE,4658000mN,CG,19T,0.042172,19TCG3058,19TCG35,3,0,{},5000
2,POINT (-72.68244 42.34953),91933,690000mE,4691000mN,XM,18T,400000.000000,18TXM9091,18TXM99,7,0,{},5000
3,POINT (-72.68240 42.34960),91933,690000mE,4691000mN,XM,18T,400000.000000,18TXM9091,18TXM99,7,0,{},5000
4,POINT (-71.17170 42.27463),106511,320000mE,4682000mN,CG,19T,0.042251,19TCG2082,19TCG28,7,0,{},5000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,POINT (-72.00956 44.44579),36867,737000mE,4925000mN,YQ,18T,275703.837111,18TYQ3725,18TYQ32,10,0,{},5000
341,POINT (-71.48081 44.90445),73302,304000mE,4975000mN,CK,19T,0.043321,19TCK0475,19TCK07,7,0,{},5000
342,POINT (-71.55434 44.48623),75409,296000mE,4929000mN,BK,19T,0.043140,19TBK9629,19TBK92,3,0,{},5000
343,POINT (-73.80366 42.65597),12855,598000mE,4723000mN,WN,18T,400000.000000,18TWN9823,18TWN92,4,0,{},5000


In [89]:
n = hosp.shape[0]
t = 20

In [90]:
ind = dict()

In [91]:
for i, r in hosp.iterrows():
    ind[r["MGRS"]] = i

In [39]:
sp_mat = np.load("sp_mat.npy")

In [40]:
sp_mat

array([[  0.        , 163.30316742,  78.05882353, ..., 338.10148675,
        141.2863607 , 231.8356676 ],
       [163.30316742,   0.        , 166.50790386, ..., 297.0678733 ,
        257.23594053, 218.5751634 ],
       [ 78.05882353, 166.50790386,   0.        , ..., 268.3956044 ,
        117.02521008, 162.12978525],
       ...,
       [338.10148675, 297.0678733 , 268.3956044 , ...,   0.        ,
        310.27608992, 110.92034763],
       [141.2863607 , 257.23594053, 117.02521008, ..., 310.27608992,
          0.        , 205.9449113 ],
       [231.8356676 , 218.5751634 , 162.12978525, ..., 110.92034763,
        205.9449113 ,   0.        ]])

In [158]:
add_to_pq = []
for i in range (t):
    loc = truck_loc[i]
    sp = nx.single_source_dijkstra_path_length(graph, loc)
    
    min_dist = np.inf
    min_loc = None
    min_j = None
    for j in range (n):
        try:
            d = sp[hosp.loc[j]["MGRS"]]
        except:
            continue
        if sp[hosp.loc[j]["MGRS"]] < min_dist:
            min_dist = sp[hosp.loc[j]["MGRS"]]
            min_loc = hosp.loc[j]["MGRS"]
            min_j = j
    
    if min_dist != np.inf:
        add_to_pq.append([min_dist, min_loc, i, min_j])

In [131]:
pq = queue.PriorityQueue()

for i in range (t):
    loc = truck_loc[i]
    sp = nx.single_source_dijkstra_path_length(graph, loc)
    
    min_dist = np.inf
    min_loc = None
    min_j = None
    for j in range (n):
        try:
            d = sp[hosp.loc[j]["MGRS"]]
        except:
            continue
        if sp[hosp.loc[j]["MGRS"]] < min_dist:
            min_dist = sp[hosp.loc[j]["MGRS"]]
            min_loc = hosp.loc[j]["MGRS"]
            min_j = j
    
    if min_dist != np.inf:
        pq.put([min_dist, min_loc, i, min_j])
        hosp.loc[min_j]["incoming"].add(i)

In [154]:
truck_loc = []
truck_supp = []
truck_timetaken = [] #in minutes
truck_path = []

max_time = 7200
for i in range (6):
    truck_loc.append("19TCG1752")
    truck_path.append(["19TCG1752"])
for i in range (7):
    truck_loc.append("18TYM0374")
    truck_path.append(["18TYM0374"])
for i in range (7):
    truck_loc.append("19TBK9604")
    truck_path.append(["19TBK9604"])
    
for i in range (20):
    truck_supp.append(10000)
    truck_timetaken.append(0)

In [165]:
hosp = joined_loc.copy()

hosp["vis"] = 0
hosp["incoming"] = [set() for i in range (hosp.shape[0])]
hosp["supplies"] = 5000

In [166]:
pq = queue.PriorityQueue()

for item in add_to_pq:
    pq.put(item)
    hosp.loc[item[3]]["incoming"].add(item[2])

In [160]:
while pq.empty() == False:
    arr = pq.get()
    print (arr)

[6.0, '19TCG1457', 0, 131]
[6.0, '19TCG1457', 1, 131]
[6.0, '19TCG1457', 2, 131]
[6.0, '19TCG1457', 3, 131]
[6.0, '19TCG1457', 4, 131]
[6.0, '19TCG1457', 5, 131]
[15.199780611545318, '18TYM0870', 6, 123]
[15.199780611545318, '18TYM0870', 7, 123]
[15.199780611545318, '18TYM0870', 8, 123]
[15.199780611545318, '18TYM0870', 9, 123]
[15.199780611545318, '18TYM0870', 10, 123]
[15.199780611545318, '18TYM0870', 11, 123]
[15.199780611545318, '18TYM0870', 12, 123]
[26.319974143503558, '19TBK9629', 13, 342]
[26.319974143503558, '19TBK9629', 14, 342]
[26.319974143503558, '19TBK9629', 15, 342]
[26.319974143503558, '19TBK9629', 16, 342]
[26.319974143503558, '19TBK9629', 17, 342]
[26.319974143503558, '19TBK9629', 18, 342]
[26.319974143503558, '19TBK9629', 19, 342]


In [167]:
while pq.empty() == False:
    arr = pq.get()
    time = arr[0]
    loc = arr[1]
    truck = arr[2]
    hosp_ind = arr[3]
    
    print (arr)
        
    if hosp.loc[hosp_ind]["vis"] == 1:
        continue
    
    
    #update this truck's path
    
    hosp.loc[hosp_ind, "vis"] = 1
    truck_loc[truck] = loc
    truck_timetaken[truck] += time
    truck_supp[truck] -= hosp.loc[hosp_ind]["supplies"]
    truck_timetaken[truck] += ((hosp.loc[hosp_ind]["supplies"] + 1999) // 2000) * 20
    truck_path[truck].append(loc)
    
        
    # reroute all trucks that had added this hospital
    for i in hosp.loc[hosp_ind]["incoming"]:
        print (i)
        loc = truck_loc[i]
    
        min_dist = np.inf
        min_loc = None
        min_j = None
        for j in range (n):
            try:
                d = sp_mat[ind[loc]]
            except:
                print (j)
                continue
            
            if hosp.loc[j]["vis"] == 0 and sp_mat[ind[loc]][j] < min_dist:
                min_dist = sp_mat[ind[loc]][j]
                min_loc = joined_loc.loc[j]["MGRS"]
                min_j = j

        if min_dist != np.inf:
            pq.put([min_dist, min_loc, i, min_j])
            print ([min_dist, min_loc, i, min_j])
            
            hosp.loc[min_j]["incoming"].add(i)


[6.0, '19TCG1457', 0, 131]
0
[13.38655462184874, '19TCG1145', 0, 68]
1
2
3
4
5
[6.0, '19TCG1457', 1, 131]
[6.0, '19TCG1457', 2, 131]
[6.0, '19TCG1457', 3, 131]
[6.0, '19TCG1457', 4, 131]
[6.0, '19TCG1457', 5, 131]
[13.38655462184874, '19TCG1145', 0, 68]
0
[7.605042016806722, '19TCG0444', 0, 299]
[7.605042016806722, '19TCG0444', 0, 299]
0
[6.605042016806723, '19TCG0238', 0, 182]
[6.605042016806723, '19TCG0238', 0, 182]
0
[2.1764705882352944, '19TCG0036', 0, 184]
[2.1764705882352944, '19TCG0036', 0, 184]
0
[2.5384615384615383, '19TCG0235', 0, 186]
[2.5384615384615383, '19TCG0235', 0, 186]
0
[5.538461538461538, '19TBG9734', 0, 76]
[5.538461538461538, '19TBG9734', 0, 76]
0
[0.0, '19TBG9734', 0, 77]
[0.0, '19TBG9734', 0, 77]
0
[2.0, '19TBG9536', 0, 187]
[2.0, '19TBG9536', 0, 187]
0
[5.0, '19TBG9931', 0, 173]
[5.0, '19TBG9931', 0, 173]
0
[0.0, '19TBG9931', 0, 174]
[0.0, '19TBG9931', 0, 174]
0
[0.0, '19TBG9931', 0, 175]
[0.0, '19TBG9931', 0, 175]
0
[1.0, '19TBG9831', 0, 176]
[1.0, '19TBG9831'

In [163]:
truck_path

[['19TCG1752',
  '19TCG1457',
  '19TCG1145',
  '19TCG0444',
  '19TCG0238',
  '19TCG0036',
  '19TCG0235',
  '19TBG9734',
  '19TBG9734',
  '19TBG9536',
  '19TBG9931',
  '19TBG9931',
  '19TBG9931',
  '19TBG9831',
  '19TCG0329',
  '19TBG9524',
  '19TBG9320',
  '19TCG1917',
  '19TCG2119',
  '19TCG2119',
  '19TCG3415',
  '19TCG3810',
  '19TCG3822',
  '19TCG4139',
  '19TCG4140',
  '19TCG2641',
  '19TCG2542',
  '19TCG3058',
  '19TCG2962',
  '19TCG2763',
  '19TCG2667',
  '19TCG2471',
  '19TCG1873',
  '19TCG1675',
  '19TCG1881',
  '19TCG2082',
  '19TCG2285',
  '19TCG2485',
  '19TCG2484',
  '19TCG2685',
  '19TCG2587',
  '19TCG2688',
  '19TCG2688',
  '19TCG2689',
  '19TCG2689',
  '19TCG2689',
  '19TCG2689',
  '19TCG2689',
  '19TCG2590',
  '19TCG2390',
  '19TCG2390',
  '19TCG2390',
  '19TCG2493',
  '19TCG2693',
  '19TCG2693',
  '19TCG2695',
  '19TCG2699',
  '19TCH2503',
  '19TCH2203',
  '19TCH1906',
  '19TCH1308',
  '19TCH0402',
  '19TCG1493',
  '19TCG1493',
  '19TCG1488',
  '19TCG1582',
  '19TCG07

In [164]:
truck_timetaken

[10604.919577154873,
 0,
 0,
 0,
 0,
 0,
 12409.909404321164,
 0,
 0,
 0,
 0,
 0,
 0,
 4759.983493630551,
 0,
 0,
 0,
 0,
 0,
 0]

In [153]:
truck_supp

[-710000,
 10000,
 10000,
 10000,
 10000,
 10000,
 -750000,
 10000,
 10000,
 10000,
 10000,
 10000,
 10000,
 -245000,
 10000,
 10000,
 10000,
 10000,
 10000,
 10000]

In [42]:
for i in range (t):
    loc = truck_loc[i]
    
    min_dist = np.inf
    min_loc = None
    for j in range (n):
        if sp_mat[ind[loc]][j] < min_dist:
            min_dist = sp_mat[ind[loc]][j]
            min_loc = joined_loc.loc[j]["MGRS"]
    
    if min_dist != np.inf:
        pq.put([-min_dist, min_loc])

KeyError: '19TCG1752'

In [43]:
ind['19TCG1752']

KeyError: '19TCG1752'